In [1]:
import pandas as pd
import numpy as np 
from preprocess import Preprocessor
import os

In [2]:
df = pd.read_csv('data/titanic.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Age,Sex,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,22.0,male,1,0,7.2500,N,S
1,2,1,1,38.0,female,1,0,71.2833,C,C
2,3,1,3,26.0,female,0,0,7.9250,N,S
3,4,1,1,35.0,female,1,0,53.1000,C,S
4,5,0,3,35.0,male,0,0,8.0500,N,S


In [4]:
n_cols = len(df.columns)
df.columns = ['id'] + ['y'] + ['x' + str(i) for i in range(n_cols - 2)]
df.columns


Index(['id', 'y', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7'], dtype='object')

In [5]:
processed_df = Preprocessor().fit_transform(df)
processed_df

,id,y,x0,x1,x2,x3,x4,x5,x6,x7
0,1,0,3,22.0,1,1,0,7.2500,7,2
1,2,1,1,38.0,0,1,0,71.2833,2,0
2,3,1,3,26.0,0,0,0,7.9250,7,2
3,4,1,1,35.0,0,1,0,53.1000,2,2
4,5,0,3,35.0,1,0,0,8.0500,7,2
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,1,0,0,13.0000,7,2
887,888,1,1,19.0,0,0,0,30.0000,1,2
888,889,0,3,24.0,0,1,2,23.4500,7,2
889,890,1,1,26.0,1,0,0,30.0000,2,0


In [18]:
processed_df.to_csv("data/processed_titanic.csv")

In [6]:
num_data = processed_df.shape[0]
train_ratio = 0.2
train_df = processed_df[int(np.ceil(num_data * train_ratio)):]
valid_df = processed_df[:int(np.ceil(num_data * train_ratio))]

In [38]:
train_df.to_csv('data/titanic_train.csv', index=False)
valid_df.to_csv('data/titanic_valid.csv', index=False)

### Data split

In [7]:
# from tqdm import tqdm
# folders = ["8_0", "6_2", "4_4", "2_6", "0_8"]

# for folder in tqdm(folders):
    
#     if not os.path.exists(os.path.join("data", folder)):
#         os.mkdir(os.path.join("data", folder))
        
#     pos = int(folder.split("_")[0])
#     train_guest_df = train_df[train_df.columns[: pos + 2]]
#     train_host_df = train_df[train_df.columns[pos + 2:]]
    
#     valid_guest_df = valid_df[valid_df.columns[: pos + 2]]
#     valid_host_df = valid_df[valid_df.columns[pos + 2:]]

#     train_guest_df.to_csv(os.path.join("data", folder, 'train_guest.csv'), index=False)
#     train_host_df.to_csv(os.path.join("data", folder, 'train_host.csv'), index=False)
#     valid_guest_df.to_csv(os.path.join("data", folder, 'valid_guest.csv'), index=False)
#     valid_host_df.to_csv(os.path.join("data", folder, 'valid_host.csv'), index=False)
    
    

In [8]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  config  pipeline config tool
  init     - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` in 127.0.0.1:9380(defaults in standalone), then exec

In [9]:
!pipeline init --ip 127.0.0.1 --port 9380

Pipeline configuration succeeded.


In [10]:
pwd

'/data/projects/fate/persistence/titanic'

### upload data

In [11]:
import os
from pipeline.backend.pipeline import PipeLine

pipeline_upload = PipeLine().set_initiator(role='guest', party_id=9999).set_roles(guest=9999)
partition = 4

dense_data_train = {"name": "titanic_train", "namespace": f"experiment"}
dense_data_valid = {"name": "titanic_valid", "namespace": f"experiment"}


data_base = "/data/projects/fate/persistence/titanic/data"
pipeline_upload.add_upload_data(file=os.path.join(data_base, "titanic_train.csv"),
                                table_name=dense_data_train["name"],             # table name
                                namespace=dense_data_train["namespace"],         # namespace
                                head=1, partition=partition)               # data info

pipeline_upload.add_upload_data(file=os.path.join(data_base, "titanic_valid.csv"),
                                table_name=dense_data_valid["name"],             # table name
                                namespace=dense_data_valid["namespace"],         # namespace
                                head=1, partition=partition)               # data info



pipeline_upload.upload(drop=1)

/data/projects/python/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-25 09:35:11.691 | ERROR    | __main__:<module>:24 - An error has been caught in function '<module>', process 'MainProcess' (703), thread 'MainThread' (140628431836992):
Traceback (most recent call last):

  File "/data/projects/fate/fate/python/fate_client/pipeline/utils/invoker/job_submitter.py", line 61, in upload_data
    raise ValueError

ValueError


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/opt/python3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ip

ValueError: job submit failed, err msg: {'retcode': 100, 'retmsg': 'Connection refused, Please check if the fate flow service is started'}